In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
dp_no_table = pd.read_csv('dp_code_match_table.csv', dtype={"department_code": "string", "department_no": "string"})
dp_no_table

In [ ]:
data = pd.read_csv('data_cleaned.csv', dtype={"std_no" : "string","department_no": "string","class_code": "string","class_name": "string","period": "string"})
data

In [ ]:
def get_department_records(dep_code):
    """verilen bölüm kısaltmasının ders kayıtlarını içeren tablo döndürür"""
    return data[data.department_no == dp_no_table[dp_no_table.department_code == dep_code].reset_index().at[0, 'department_no']].groupby('std_no').class_name.apply(list).reset_index()
def get_class_code_records_as_list(df):
    """verilen tablonun class_code sütunundaki değeleri liste olarak döndürür"""
    return df.class_name.to_list()

In [ ]:
blm_records = get_department_records('BLM')
blm_records

In [ ]:
blm_records_class_codes_list = get_class_code_records_as_list(blm_records)
te = TransactionEncoder()
blm_records_oneshot = te.fit(blm_records_class_codes_list).transform(blm_records_class_codes_list)
blm_records_oneshot

In [ ]:
blm_records_oneshot_df = pd.DataFrame(blm_records_oneshot, columns=te.columns_)
blm_records_oneshot_df

In [ ]:
supports = {}
for col in blm_records_oneshot_df.columns:
    supports[col] = blm_records_oneshot_df[col].value_counts()[1] / blm_records_oneshot_df.shape[0]

fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([0,0,1,1])
ax.bar(x=list(supports.keys()), height=list(supports.values()))
plt.show()

In [ ]:
freq_itemsets = apriori(blm_records_oneshot_df, min_support=0.01, use_colnames=True)
freq_itemsets['length'] = freq_itemsets.itemsets.apply(lambda x: len(x))
freq_itemsets

In [ ]:
assoc_rules = association_rules(freq_itemsets, metric="lift", min_threshold=1)
#assoc_rules = assoc_rules[(assoc_rules['confidence'] > 0.3) & (assoc_rules['antecedent support'] > 0.05)]
#assoc_rules = assoc_rules[(assoc_rules['confidence'] > 0.8)]
assoc_rules = assoc_rules.sort_values('confidence', ascending=False)
assoc_rules = assoc_rules.reset_index().drop('index', axis=1)
assoc_rules["antecedents"] = assoc_rules["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
assoc_rules["consequents"] = assoc_rules["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
assoc_rules = assoc_rules[assoc_rules["consequents"].str.contains(',') == False]
assoc_rules

In [ ]:
##gelen kural sayısına göre dinamik olarak bazı değerler değişecek
##hedef 2 ila 10 kural arası
assoc_rules[assoc_rules['antecedents'] == 'Yapay Zeka']

In [ ]:
def create_rules(df, antecedents):
    min_conf = 0.1
    delta = min_conf / 4
    rules = df[(df['antecedents'] == antecedents) & (df['confidence'] > min_conf)]
    ##satır sayısı 3'den azsa confidence'i azalt
    while rules.shape[0] < 3 or rules.shape[0] > 10:
        if rules.shape[0] < 3:
            min_conf -= delta
            rules = df[(df['antecedents'] == antecedents) & (df['confidence'] > min_conf)]
        
        if rules.shape[0] > 10:
            min_conf += delta
            rules = df[(df['antecedents'] == antecedents) & (df['confidence'] > min_conf)]
    return rules

In [ ]:
create_rules(assoc_rules, 'Yapay Zeka')